In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
print(os.listdir('/content/drive/My Drive'))


['Classroom', 'zoom_0 (1).mp4', 'zoom_0.mp4', 'ömer writing 1 (1).docx', 'ömer weekly translation exercise modals 1,2.docx', 'ömer adjectives adverbs örnek cümleler.docx', 'ömer weekly translation passive exercise 1-2.docx', 'ömer weekly translation exercises relative clause.docx', 'ömer weekly translation exercises noun clause.docx', 'ömer weekly translation exercises if clauses.docx', 'ömer writing 1.docx', 'ömer writing 2.docx', 'ömer writing 3.docx', 'ömer writing 4.gdoc', 'ömer writing 5.docx', 'Kardeslere Ait Bilgiler.docx', 'Screenshot 2021-09-09 113901.png', 'yok-ogrenci-belgesi-sorgulama (1).pdf', 'nvi-nufus-aile kayit-ornegi-belgesi-sorgulama.pdf', 'adli-sicil-kaydi (1).pdf', 'Başlıksız e-tablo (1).gsheet', 'Başlıksız e-tablo.gsheet', 'p_transcript (1).pdf', 'uc.png', 'WhatsApp Image 2022-05-17 at 18.06.31.jpeg', 'fizik 2', '75448869_132391081083378_1851550492647236146_n.jpg adlı dosyanın kopyası (1)', '75448869_132391081083378_1851550492647236146_n.jpg adlı d

In [ ]:
print(os.listdir('/content/drive/My Drive/Colab_Data'))


['train_feats.npy', 'train_labels.csv', 'valtest_feats.npy', 'submission.csv', 'submission_Deep_Dense_Dropout_BN_L2_f1_0.98863.csv', 'submission_Improved_Deep_Dense_L2_BN_Ensemble_f1_0.98727.csv']


In [ ]:
import numpy as np
import pandas as pd

# Dosya yollarını belirtin
train_feats_path = '/content/drive/My Drive/Colab_Data/train_feats.npy'
train_labels_path = '/content/drive/My Drive/Colab_Data/train_labels.csv'
valtest_feats_path = '/content/drive/My Drive/Colab_Data/valtest_feats.npy'

# Dosyaları yükleyin
train_feats = np.load(train_feats_path, allow_pickle=True).item()
train_labels = pd.read_csv(train_labels_path)
valtest_feats = np.load(valtest_feats_path, allow_pickle=True).item()

print("Dosyalar başarıyla yüklendi!")


Dosyalar başarıyla yüklendi!


In [ ]:
print("train_feats keys:", train_feats.keys())
print("train_labels head:", train_labels.head())
print("valtest_feats keys:", valtest_feats.keys())


train_feats keys: dict_keys(['idx', 'resnet_feature', 'vit_feature', 'clip_feature', 'dino_feature'])
train_labels head:    ID  label
0   0      1
1   1      3
2   2      8
3   3      7
4   4      9
valtest_feats keys: dict_keys(['idx', 'resnet_feature', 'vit_feature', 'clip_feature', 'dino_feature'])


In [ ]:
pip install dask[dataframe]

INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 8.8 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade tensorflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 110.2 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.1
    Uninstalling tensorflow-2.17.1:
      Successfully uninstalled tensorflow-2.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, Callback
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from google.colab import drive


drive.mount('/content/drive')


train_feats_path = '/content/drive/My Drive/Colab_Data/train_feats.npy'
train_labels_path = '/content/drive/My Drive/Colab_Data/train_labels.csv'
valtest_feats_path = '/content/drive/My Drive/Colab_Data/valtest_feats.npy'


train_feats = np.load(train_feats_path, allow_pickle=True).item()
train_labels = pd.read_csv(train_labels_path)
valtest_feats = np.load(valtest_feats_path, allow_pickle=True).item()

resnet_features = train_feats['resnet_feature']
vit_features = train_feats['vit_feature']
clip_features = train_feats['clip_feature']
dino_features = train_feats['dino_feature']

X = np.hstack([resnet_features, vit_features, clip_features, dino_features])
y = train_labels['label'].values

valtest_resnet_features = valtest_feats['resnet_feature']
valtest_vit_features = valtest_feats['vit_feature']
valtest_clip_features = valtest_feats['clip_feature']
valtest_dino_features = valtest_feats['dino_feature']
X_test = np.hstack([valtest_resnet_features, valtest_vit_features, valtest_clip_features, valtest_dino_features])

def mixup_data(X, y, alpha=0.3):
    indices = np.random.permutation(len(X))
    X_shuffled = X[indices]
    y_shuffled = y[indices]
    lam = np.random.beta(alpha, alpha)
    X_mix = lam * X + (1 - lam) * X_shuffled

    return X_mix, y, lam, y_shuffled

class F1Callback(Callback):
    def __init__(self, X_val, y_val):
        super().__init__()
        self.X_val = X_val
        self.y_val = y_val
        self.best_f1 = 0.0

    def on_epoch_end(self, epoch, logs=None):
        y_pred_proba = self.model.predict(self.X_val)
        y_pred = np.argmax(y_pred_proba, axis=1)
        f1_val = f1_score(self.y_val, y_pred, average='macro')
        logs['val_f1'] = f1_val
        print(f"Epoch {epoch+1} => val_f1: {f1_val:.5f}")

        if f1_val > self.best_f1:
            self.best_f1 = f1_val
            print(f"Yeni en iyi F1 => {f1_val:.5f}, model kaydedildi.")
            self.model.save_weights("best_f1_model.weights.h5")


lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=800,
    decay_rate=0.95,
    staircase=False
)


def create_model(input_dim):
    model = Sequential()

    model.add(Dense(1024, activation='relu', kernel_regularizer=l2(0.00005), input_shape=(input_dim,)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(512, activation='relu', kernel_regularizer=l2(0.00005)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.00005)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.00005)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(10, activation='softmax'))

    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


SEED_LIST = [42, 2023, 777, 999, 314159, 2024, 1234]  # 7 seeds
FOLDS = 5

oof_preds = np.zeros((len(X), len(SEED_LIST)*FOLDS, 10))
test_preds = np.zeros((len(X_test), len(SEED_LIST)*FOLDS, 10))

meta_col_index = 0

for seed_val in SEED_LIST:
    print(f"\n========================== SEED = {seed_val} ==========================")
    np.random.seed(seed_val)
    tf.random.set_seed(seed_val)

    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=seed_val)

    for fold_index, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        print(f"\n--- Fold {fold_index+1} --- (Seed {seed_val})")

        X_train_cv, X_val_cv = X[train_idx], X[val_idx]
        y_train_cv, y_val_cv = y[train_idx], y[val_idx]


        scaler = StandardScaler()
        X_train_cv = scaler.fit_transform(X_train_cv)
        X_val_cv = scaler.transform(X_val_cv)


        X_train_mix, y_train_mix, _, _ = mixup_data(X_train_cv, y_train_cv, alpha=0.3)

        model = create_model(X_train_cv.shape[1])

        f1_cb = F1Callback(X_val_cv, y_val_cv)
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

        model.fit(
            X_train_mix, y_train_mix,
            validation_data=(X_val_cv, y_val_cv),
            epochs=50,
            batch_size=64,
            callbacks=[f1_cb, early_stopping],
            verbose=1
        )

        model.load_weights("best_f1_model.weights.h5")


        val_proba = model.predict(X_val_cv)
        oof_preds[val_idx, meta_col_index, :] = val_proba


        X_test_scaled = scaler.transform(X_test)
        test_proba = model.predict(X_test_scaled)
        test_preds[:, meta_col_index, :] = test_proba

        meta_col_index += 1


oof_reshaped = oof_preds.reshape(len(X), -1)
test_reshaped = test_preds.reshape(len(X_test), -1)

meta_model = LogisticRegression(
    penalty='l2',
    C=3.0,
    solver='lbfgs',
    max_iter=2000
)
meta_model.fit(oof_reshaped, y)

oof_pred_meta = meta_model.predict(oof_reshaped)
f1_ensemble = f1_score(y, oof_pred_meta, average='macro')
print("\nFinal Ensemble OOF F1 =>", f1_ensemble)

test_pred_meta = meta_model.predict(test_reshaped)
submission_df = pd.DataFrame({"ID": range(len(test_pred_meta)), "Predicted": test_pred_meta})
submission_name = f"/content/drive/My Drive/Colab_Data/submission_ensemble_stacking_F1_{f1_ensemble:.5f}.csv"
submission_df.to_csv(submission_name, index=False)
print(f"\nSubmission kaydedildi => {submission_name}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

========================== SEED = 42 ==========================

--- Fold 1 --- (Seed 42)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Epoch 1 => val_f1: 0.98314
Yeni en iyi F1 => 0.98314, model kaydedildi.
500/500 ━━━━━━━━━━━━━━━━━━━━ 13s 11ms/step - accuracy: 0.9304 - loss: 0.3737 - val_accuracy: 0.9831 - val_loss: 0.1999 - val_f1: 0.9831
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 2 => val_f1: 0.98298
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9852 - loss: 0.1934 - val_accuracy: 0.9830 - val_loss: 0.2088 - val_f1: 0.9830
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Epoch 3 => val_f1: 0.98300
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9903 - loss: 0.1706 - val_accuracy: 0.9830 - val_loss: 0.1946 - val_f1: 0.9830
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Epoch 4 => val_f1: 0.98402
Yeni en iyi F1 => 0.98402, model kaydedildi.
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9937 - loss: 0.1519 - val_accuracy: 0.9840 - val_loss: 0.1936 - val_f1: 0.9840
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Epoc

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Epoch 1 => val_f1: 0.97689
Yeni en iyi F1 => 0.97689, model kaydedildi.
500/500 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - accuracy: 0.7240 - loss: 0.9456 - val_accuracy: 0.9769 - val_loss: 0.2160 - val_f1: 0.9769
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Epoch 2 => val_f1: 0.97929
Yeni en iyi F1 => 0.97929, model kaydedildi.
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.8526 - loss: 0.5427 - val_accuracy: 0.9793 - val_loss: 0.2230 - val_f1: 0.9793
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 3 => val_f1: 0.98002
Yeni en iyi F1 => 0.98002, model kaydedildi.
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.8933 - loss: 0.4498 - val_accuracy: 0.9800 - val_loss: 0.2330 - val_f1: 0.9800
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 4 => val_f1: 0.98026
Yeni en iyi F1 => 0.98026, model kaydedildi.
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9162 - loss: 0.3974 - val_accuracy: 0.9803 - 

...

Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Epoch 1 => val_f1: 0.98200
Yeni en iyi F1 => 0.98200, model kaydedildi.
500/500 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.8747 - loss: 0.5480 - val_accuracy: 0.9820 - val_loss: 0.2097 - val_f1: 0.9820
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 2 => val_f1: 0.98388
Yeni en iyi F1 => 0.98388, model kaydedildi.
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9606 - loss: 0.2679 - val_accuracy: 0.9839 - val_loss: 0.2118 - val_f1: 0.9839
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 3 => val_f1: 0.98115
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9774 - loss: 0.2193 - val_accuracy: 0.9811 - val_loss: 0.2193 - val_f1: 0.9811
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 4 => val_f1: 0.98203
500/500 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.9808 - loss: 0.2118 - val_accuracy: 0.9820 - val_loss: 0.2315 - val_f1: 0.9820
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoc

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Epoch 1 => val_f1: 0.25316
Yeni en iyi F1 => 0.25316, model kaydedildi.
500/500 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.1408 - loss: 2.7183 - val_accuracy: 0.2860 - val_loss: 1.8621 - val_f1: 0.2532
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 2 => val_f1: 0.31098
Yeni en iyi F1 => 0.31098, model kaydedildi.
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.3986 - loss: 1.8176 - val_accuracy: 0.3115 - val_loss: 1.7085 - val_f1: 0.3110
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 3 => val_f1: 0.31159
Yeni en iyi F1 => 0.31159, model kaydedildi.
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.5551 - loss: 1.3864 - val_accuracy: 0.3234 - val_loss: 1.7323 - val_f1: 0.3116
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Epoch 4 => val_f1: 0.41554
Yeni en iyi F1 => 0.41554, model kaydedildi.
500/500 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.6447 - loss: 1.1644 - val_accuracy: 0.4149 - 

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Epoch 1 => val_f1: 0.98250
Yeni en iyi F1 => 0.98250, model kaydedildi.
500/500 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.9355 - loss: 0.3539 - val_accuracy: 0.9825 - val_loss: 0.2009 - val_f1: 0.9825
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 2 => val_f1: 0.98499
Yeni en iyi F1 => 0.98499, model kaydedildi.
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9865 - loss: 0.1874 - val_accuracy: 0.9850 - val_loss: 0.1914 - val_f1: 0.9850
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 3 => val_f1: 0.98463
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9913 - loss: 0.1641 - val_accuracy: 0.9846 - val_loss: 0.1893 - val_f1: 0.9846
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Epoch 4 => val_f1: 0.98177
500/500 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.9942 - loss: 0.1485 - val_accuracy: 0.9818 - val_loss: 0.2005 - val_f1: 0.9818
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoc